In [ ]:
#author: ML Tlachac

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import textblob as tb
import json

#load text message data
df = pd.read_csv("textDF.csv")
print("Number of Texts: " + str(df.shape[0]))
print("Number of Participants: " + str(len(set(df.id))))
df1 = df[df["type.1"] == 1]
df2 = df[df["type.1"] == 2]
print("Number of Recieved Texts: " + str(df1.shape[0]))
print("Number of Participants with Recieved Texts: " + str(len(set(df1.id))))
print("Number of Sent Texts: " + str(df2.shape[0]))
print("Number of Participants with Sent Texts: " + str(len(set(df2.id))))

In [ ]:
#set of IDs
setID1 = list(set(df1.id))
setID2 = list(set(df2.id))

#Recieved Texts information extraction
pID = []
pAddress = []
pContent = []
nTexts = []
pDF1 = pd.DataFrame()
for i in range(0, len(setID1)):
    tempDF = df1[df1.id == setID1[i]]
    addressID = list(set(tempDF.address))
    for j in range(0, len(addressID)):
        pID.append(setID1[i])
        pAddress.append(addressID[j])
        tempDF2 = tempDF[tempDF.address == addressID[j]]
        tempDF2 = tempDF2.set_index(np.arange(0,tempDF2.shape[0]))
        texts = []
        for k in range(0, tempDF2.shape[0]):
            texts.append(tempDF2.body[k])
        pContent.append(texts)
        nTexts.append(len(texts))
pDF1["ID"] = pID
pDF1["Address"] = pAddress
pDF1["Content"] = pContent
pDF1["Texts"] = nTexts
print("Number of Participant-Sender Combinations for Recieved Texts: " + str(pDF1.shape[0]))

In [ ]:
#text message POS tags and sentiment

polarity = []
subjectivity = []
tags = []
for i in range(0, len(pDF1.ID)):
    print(i)
    polarity2 = []
    subjectivity2 = []
    tags2 = []
    for text in pDF1.Content[i]:
        T = TextBlob(str(text))
        polarity2.append(T.sentiment[0])
        subjectivity2.append(T.sentiment[1])
        for word, tag in T.tags:
            tags2.append(tag)
    tags.append(tags2)
    polarity.append(polarity2)
    subjectivity.append(subjectivity2)
pDF1["POStags"] = tags
pDF1["Polarity"] = polarity
pDF1["Subjectivity"] = subjectivity

In [ ]:
#calculate features from POStags column
posTags = []
for i in range(0, pDF1.shape[0]):
    for tag in pDF1.POStags[i]:
        posTags.append(tag)
for i in range(0, pDF2.shape[0]):
    for tag in pDF2.POStags[i]:
        posTags.append(tag)
for i in range(0, pDFt.shape[0]):
    for tag in pDFt.POStags[i]:
        posTags.append(tag)
posSet = set(posTags)
print(posSet)

poswords1 = []
for posList in pDF1.POStags:
    poswords1.append(len(posList))
pDF1["WordsTags"] = poswords1

for tag in posSet:
    cnt = []
    for posList in pDF1.POStags:
        counter = 0
        for item in posList:
            if item == tag:
                counter += 1
        cnt.append(counter)
    pDF1[tag] = cnt

In [ ]:
#sentiment features for text messages

pcount = []
ncount = []
pavg = []
navg = []
scount = []
savg = []
for i in range(0, pDF1.shape[0]):
    s = []
    p = []
    n = []
    for item in pDF1.Polarity[i]:
        if item > 0:
            p.append(item)
        if item < 0:
            n.append(item)
    for item in pDF1.Subjectivity[i]:
        if item > 0:
            s.append(item)
    pcount.append(len(p))
    ncount.append(len(n))
    scount.append(len(s))
    if len(p) > 0:
        pavg.append(sum(p)/len(p))
    else:
        pavg.append(0)
    if len(n) > 0:
        navg.append(sum(n)/len(n))
    else:
        navg.append(0)
    if len(s) > 0:
        savg.append(sum(s)/len(s))
    else:
        savg.append(0)
pDF1["PositiveCnt"] = pcount
pDF1["NegativeCnt"] = ncount
pDF1["PostitiveAvg"] = pavg
pDF1["NegativeAvg"] = navg
pDF1["SubjectiveCnt"] = scount
pDF1["SubjectiveAvg"] = savg

In [ ]:
#save pDF1 to be used in FeatureSetsForIEEEBHI2019.ipynb
pDF1.to_csv("TextSetupDFa.csv")